In [1]:
import pandas as pd
from elasticsearch import Elasticsearch, helpers

In [2]:
# 정책, 용어 파일 로드
policy_data = pd.read_excel('origin_policy_chunk.xlsx')
words_data = pd.read_excel('origin_word_chunk.xlsx')

In [3]:
policy_data.head()

,rnum,정책
0,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...
1,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...
2,1,지역: 노원구\n분야: 일자리\n정책명: 청년 평생 교육 프로그램 ‘다 담다’ 운영...
3,2,지역: 은평구\n분야: 일자리\n정책명: 청년 창업점포 운영\n정책 설명: 청년 창...
4,2,지역: 은평구\n분야: 일자리\n정책명: 청년 창업점포 운영\n정책 설명: 청년 창...


In [4]:
words_data.head()

,rnum,용어
0,1,디지털 드로잉은 컴퓨터나 태블릿과 같은 디지털 기기를 사용하여 그림을 그리는 것을 ...
1,2,만 나이는 사람이 실제로 살아온 정확한 연수를 기준으로 계산하는 나이입니다. 한국에...
2,3,"비영리단체는 사회적, 교육적, 자선적, 문화적 또는 과학적 목적을 추구하기 위해 설..."
3,4,협동조합은 공동의 목표를 가진 회원들이 자발적으로 모여 운영하는 비즈니스 조직입니다...
4,5,새싹점포 청년상인은 전통시장이나 상점가에서 새롭게 사업을 시작하는 젊은 상인들을 의...


In [8]:
# Elasticsearch 연결 설정
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme':'http'}], http_auth=('elastic', 'BD_pl-B5*iWsE+Jwwoqr') )

In [10]:
#정책 인덱스 없으면 생성, 청크x
index_name1 = 'policy'
if not es.indices.exists(index=index_name1):
    es.indices.create(index=index_name1)

# 데이터 인덱싱
def generate_docs(data):
    for idx, row in data.iterrows():
        yield {
            "_index": index_name1,
            "_id": row['rnum'],
            "_source": {
                "text": row['text'],
                "embedding": row['embeddings'].tolist()
            }
        }

helpers.bulk(es, generate_docs(policy_data))

(117, [])

In [ ]:
#용어 인덱스 없으면 생성, 청크x
index_name2 = 'word'
if not es.indices.exists(index=index_name2):
    es.indices.create(index=index_name2)

# 데이터 인덱싱
def generate_docs(data):
    for idx, row in data.iterrows():
        yield {
            "_index": index_name2,
            "_id": row['rnum'],
            "_source": {
                "text": row['text'],
                "embedding": row['embeddings'].tolist()
            }
        }

helpers.bulk(es, generate_docs(words_data))

In [11]:
# 검색 쿼리 생성 함수
def search(query, top_k=5):
    # 쿼리 텍스트 임베딩
    query_embedding = embed_text(query).tolist()

    # 검색 쿼리
    search_query = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": query_embedding}
                }
            }
        }
    }

    # 검색 실행
    response = es.search(index=index_name, body=search_query)
    return response['hits']['hits']


In [14]:
# 예제 검색
query = "청년내일저축계좌에 대해 알려줘"
results = search(query)
for result in results:
    print(result['_source']['text'], result['_score'])
    print('==============================================================')

기관/지자체 구분: 지자체
정책명: 서울시 가족돌봄청년 지원사업
정책 설명: 가족돌봄청년 상황에 맞는 맞춤형 자원 안내 및 연결
생활비, 병원비, 주거비, 학습비, 문화활동 참여 기회 등(별도 공고를 통해 신청자격, 기한, 지원내용 안내)
지원 내용: 가족돌봄청년의 상황에 맞는 자원 연결
1. 가족돌봄청년 후원자원 배분 : 생계비·학습비·의료비 등 현금 지원, 공공주택 입주자격 등
2. 가족돌봄청년 홍보 및 발굴
3. 가족돌봄청년 당사자 및 유관기관 담당자 역량강화 등
모집 기간: 운영시간 : 평일(월~금) 09:00~18:00
연령 정보: 만 9세 ~ 34세
지원 조건: 서울시에 주민등록상 거주
추가 요구 사항: 민법상 가족이 장애, 정신 신체의 질병이 있고, 본인이 가족 돌봄(가사, 간병 등) 또는 가족의 생계를 책임(전부 또는 일부)지고 있으며, 가족 돌봄으로 인해 경제적 어려움, 개인생활의 제한, 학업 또는 생계활동 유지의 어려움, 문화 여가활동의 제한 등을 겪는 경우
신청 방법: 상담을 원하는 경우
- 당사자 직접 신청(상담신청서 온라인 작성 / 전화)
- 유관기관 대리 신청(자치도 동주민센터, 학교, 병원 등)
신청 URL: https://forms.gle/ad34EcJUi5zgPXXZ7
참고 사이트 URL1: https://www.welfare.seoul.kr/web/contents/business1-3-8.do?schM=view&id=8412
관리 부서: 서울시복지재단
위탁 담당자: 서울시복지재단
기타: 서울시 가족돌봄청년 지원 전담기구
- 운영시간 : 평일(월~금) 9:00~18:00
- 장소 : 서울시 동작구 현충로 75, 6층 서울시복지재단 흑석동 별관
- 문의방법 : 전화(02-6353-0336-9) 1.8015814
기관/지자체 구분: 지자체
정책명: 청년진로탐색 워크샵 <내 길을 찾는 네 가지 질문> 참여자 모집
정책 설명: 나를 이해하기 위한 네 가지 질문을 공유하고, 답을 탐색하는 과정을 거치며 세상과 나를 연결하는 수 많은 고리들